In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
pip install peft transformers datasets bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 17.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, Trainer, TrainingArguments, BitsAndBytesConfig
from datasets import Dataset, DatasetDict
import torch

In [ ]:
import pandas as pd
from datasets import Dataset

csv_file_path = "/content/drive/MyDrive/aidms/augmented_train_dataset.csv"
df = pd.read_csv(csv_file_path)

dataset = Dataset.from_pandas(df)

print(dataset)

Dataset({
    features: ['text', 'code'],
    num_rows: 9797
})


In [ ]:
def format_examples(example):
    """
    Format examples for code completion.
    Wraps the code column into a prompt-friendly structure.
    """
    system_prompt = "<|system|>\nYou are a helpful assistant specializing in coding.<|end|>\n"
    code_prompt = f"<|user|>\nComplete the following code:<|end|>\n<|assistant|>\n{example['code']}<|end|>\n"
    return {"text": system_prompt + code_prompt}


formatted_dataset = dataset.map(format_examples, batched=False)


formatted_dataset.save_to_disk("/content/drive/MyDrive/aidms/formatted_train_dataset")

Map:   0%|          | 0/9797 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9797 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk
import pandas as pd

formatted_dataset = load_from_disk("/content/drive/MyDrive/aidms/formatted_train_dataset")

# Inspect dataset
print("Dataset Structure:")
print(formatted_dataset)

print("\nColumn Names:")
print(formatted_dataset.column_names)

print("\nNumber of Rows:")
print(len(formatted_dataset))

print("\nFirst 5 Rows:")
for row in formatted_dataset["text"][:5]:
    print(row)


Dataset Structure:
Dataset({
    features: ['text', 'code'],
    num_rows: 9797
})

Column Names:
['text', 'code']

Number of Rows:
9797

First 5 Rows:
<|system|>
You are a helpful assistant specializing in coding.<|end|>
<|user|>
Complete the following code:<|end|>
<|assistant|>
#include <bits/stdc++.h> NEW_LINE using namespace std ; int maxPresum ( vector < int > a , vector < int > b ) { int X = max ( a [ 0 ] , 0 ) ; for ( int i = 1 ; i < a . size ( ) ; i ++ ) { a [ i ] += a [ i - 1 ] ; X = max ( X , a [ i ] ) ; } int Y = max ( b [ 0 ] , 0 ) ; for ( int i = 1 ; i < b . size ( ) ; i ++ ) { b [ i ] += b [ i - 1 ] ; Y = max ( Y , b [ i ] ) ; } return X + Y ; } int main ( ) { vector < int > A = { 2 , -1 , 4 , -5 } ; vector < int > B = { 4 , -3 , 12 , 4 , -3 } ; cout << maxPresum ( A , B ) << endl ; }<|end|>

<|system|>
You are a helpful assistant specializing in coding.<|end|>
<|user|>
Complete the following code:<|end|>
<|assistant|>
#include <bits/stdc++.h> NEW_LINE using namespace std

In [ ]:
formatted_dataset = formatted_dataset.remove_columns(
    [col for col in formatted_dataset.column_names if col != "text"]
)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, PeftModel

# Load model
def load_model(model_name, cache_dir):
    compute_dtype = getattr(torch, "float16")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_name, quantization_config=bnb_config, cache_dir=cache_dir, device_map="auto"
    )
    return model

# Load tokenizer
def get_tokenizer(model_name="microsoft/Phi-3.5-mini-instruct"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    return tokenizer

# Define LoRA configuration
def get_peft_config():
    return LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules="all-linear",
    )

In [ ]:
pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 20.8 MB/s eta 0:00:00


In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Fine-tuning setup
def create_fine_tuning_trainer(model, tokenizer, train_set, peft_config):
    training_arguments = TrainingArguments(
        output_dir="/content/drive/MyDrive/aidms/mem-leak-code-gen-phi3.5-mini",
        num_train_epochs=1,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        save_steps=25,
        logging_steps=25,
        learning_rate=2e-4,
        weight_decay=0.001,
        fp16=False,
        bf16=False,
        max_grad_norm=0.3,
        warmup_ratio=0.03,
        group_by_length=True,
        lr_scheduler_type="cosine",
        report_to="tensorboard",
    )

    trainer = SFTTrainer(
        model=model,
        args=training_arguments,
        train_dataset=train_set,
        tokenizer=tokenizer,
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=min(tokenizer.model_max_length, 256),
    )
    return trainer

In [ ]:
from datasets import load_from_disk

model = load_model(model_name="microsoft/Phi-3.5-mini-instruct", cache_dir="model-cache/")
tokenizer = get_tokenizer()
peft_config = get_peft_config()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
trainer = create_fine_tuning_trainer(model, tokenizer, formatted_dataset, peft_config)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/9797 [00:00<?, ? examples/s]

Step,Training Loss
25,1.565300
50,0.970400
75,0.606700
100,0.558100
125,0.513000
150,0.508300
175,0.512000
200,0.505400
225,0.509900
250,0.455800


config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

TrainOutput(global_step=4899, training_loss=0.4471476565187964, metrics={'train_runtime': 4077.8734, 'train_samples_per_second': 2.402, 'train_steps_per_second': 1.201, 'total_flos': 4.90286085345792e+16, 'train_loss': 0.4471476565187964, 'epoch': 1.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/aidms/mem-leak-code-gen-phi3.5-mini")
print("Fine-tuning complete. Model saved!")

Fine-tuning complete. Model saved!
